In [0]:
import databricks.koalas as ks
import matplotlib.pyplot as plt
import numpy as np
from databricks.koalas.config import set_option

In [0]:
set_option("compute.ops_on_diff_frames", True)

In [0]:
# Генерация случайной выборки точек на координатной плоскости 100x100: 70% - участвуют, из них 95% - маркируются по сумме координат, а оставльные 5% - рандомно
xs = list(np.arange(0, 100)) * 100
ys = [item for sublist in list(map(lambda x: [x] * 100, np.arange(0, 100))) for item in sublist]
points = list(zip(xs, ys))
np.random.shuffle(points)
involved_points = points[:int(len(points) * 0.7)]
involved_points_95 = involved_points[:int(len(involved_points) * 0.95)]
involved_points_5 = involved_points[-int(len(involved_points) * 0.05):]

In [0]:
def label_points(fst, snd):
  if fst + snd < 30:
    return 'A'
  elif fst + snd < 65:
    return 'B'
  else:
    return 'C'
  
def random_label_points():
  rand = np.random.random()
  if rand < 1 / 3:
    return 'A'
  elif rand < 2 / 3:
    return 'B'
  else:
    return 'C'

# Маркировка данных
labeled_involved_points_95 = [(first, second, label_points(first, second)) for (first, second) in involved_points_95]
labeled_involved_points_5 = [(first, second, random_label_points()) for (first, second) in involved_points_5]
all_points = labeled_involved_points_95 + labeled_involved_points_5
knn_data = ks.DataFrame(all_points, columns=['x', 'y', 'attr'])
knn_data.cache()
knn_data.head()

/databricks/python/lib/python3.8/site-packages/databricks/koalas/frame.py:4350: FutureWarning: DataFrame.cache is deprecated as of DataFrame.spark.cache. Please use the API instead.
 warnings.warn(
Out[4]:

,x,y,attr
0,6,22,A
1,31,74,C
2,62,14,C
3,80,82,C
4,70,73,C


In [0]:
# Создание подмножеств классифицированных и некласифицированных данных
knn_classified_data = knn_data[:int(len(knn_data) * 0.9)]
knn_classified_data.cache()
knn_unclassified_data = knn_data[-int(len(knn_data) * 0.1):].drop('attr')
knn_unclassified_data.cache()
;

/databricks/python/lib/python3.8/site-packages/databricks/koalas/frame.py:4350: FutureWarning: DataFrame.cache is deprecated as of DataFrame.spark.cache. Please use the API instead.
 warnings.warn(
Out[5]: ''

In [0]:
# Создание таблицы distanced - таблицы расстояний между классифицированными и неклассифицированными точками

cartesian_product = lambda left, right: left.assign(key=1).merge(right.assign(key=1), on='key').drop('key', 1)
distanced = cartesian_product(knn_unclassified_data, knn_classified_data).rename(columns={'x_x': 'x', 'y_x': 'y', 'x_y': 'x2', 'y_y': 'y2'})
distanced['distance'] = np.sqrt((distanced.x - distanced.x2) ** 2 + (distanced.y - distanced.y2) ** 2)
distanced.cache()
distanced.head()

Out[6]:

,x,y,x2,y2,attr,distance
0,75,23,6,22,A,69.007246
1,75,23,31,74,C,67.357256
2,75,23,62,14,C,15.811388
3,75,23,80,82,C,59.211485
4,75,23,70,73,C,50.249378


In [0]:
# Добавление к полученной таблице столбца distance_rn - порядкового номера числа расстояния (по возрастанию) для текущей пары x, y (???)

distanced_row_numbered = distanced.copy()
distanced_row_numbered['distance_rn'] = distanced.groupby(by=['x', 'y'])['distance'].rank(method="first", ascending=True).astype(int)
distanced_row_numbered.cache()
distanced_row_numbered.head()

/databricks/python/lib/python3.8/site-packages/databricks/koalas/frame.py:4350: FutureWarning: DataFrame.cache is deprecated as of DataFrame.spark.cache. Please use the API instead.
 warnings.warn(
Out[7]:

,x,y,x2,y2,attr,distance,distance_rn
216768,0,4,1,4,A,1.000000,1
217384,0,4,0,5,A,1.000000,2
219456,0,4,0,3,A,1.000000,3
217607,0,4,1,5,A,1.414214,4
218543,0,4,0,6,A,2.000000,5


In [0]:
# Выбрать все записи из полученной таблицы, distance_rn которых не превышает 5
top_n_selected = distanced_row_numbered[distanced_row_numbered["distance_rn"] <= 5]
top_n_selected.head()

Out[8]:

,x,y,x2,y2,attr,distance,distance_rn
216768,0,4,1,4,A,1.000000,1
217384,0,4,0,5,A,1.000000,2
219456,0,4,0,3,A,1.000000,3
217607,0,4,1,5,A,1.414214,4
218543,0,4,0,6,A,2.000000,5


In [0]:
# Подсчет частоты встречаемости у соседей (поле attr_cnt)
top_n_attr_counted = top_n_selected[['x', 'y', 'attr']].copy()
top_n_attr_counted['attr_cnt'] = 0
top_n_attr_counted = top_n_attr_counted.groupby(by=top_n_attr_counted.columns[: -1].values, as_index=False).count()
top_n_attr_counted.cache()
top_n_attr_counted.head()

/databricks/python/lib/python3.8/site-packages/databricks/koalas/frame.py:4350: FutureWarning: DataFrame.cache is deprecated as of DataFrame.spark.cache. Please use the API instead.
 warnings.warn(
Out[9]:

,x,y,attr,attr_cnt
0,0,4,A,5
1,0,7,A,5
2,0,16,A,5
3,0,32,B,5
4,0,35,B,5


In [0]:
# Добавление к полученной таблице столбца attr_rn - порядкового номера числа количества упоминаний attr у соседей для этой пары x, y (???)
top_n_attr_counted_and_numbered = top_n_attr_counted.copy()
top_n_attr_counted_and_numbered['attr_rn'] = top_n_attr_counted_and_numbered.groupby(['x', 'y'])['attr_cnt'].rank(method="first", ascending=False)
top_n_attr_counted_and_numbered.cache()
top_n_attr_counted_and_numbered.head()

/databricks/python/lib/python3.8/site-packages/databricks/koalas/frame.py:4350: FutureWarning: DataFrame.cache is deprecated as of DataFrame.spark.cache. Please use the API instead.
 warnings.warn(
Out[10]:

,x,y,attr,attr_cnt,attr_rn
0,0,4,A,5,1.0
1,0,7,A,5,1.0
2,0,16,A,5,1.0
3,0,32,B,5,1.0
4,0,35,B,5,1.0


In [0]:
# Выбор самого часто встречаемого атрибута у соседей
classified_unclassified_data = top_n_attr_counted_and_numbered[top_n_attr_counted_and_numbered["attr_rn"] == 1][["x", "y", "attr"]]
classified_unclassified_data.head()

Out[11]:

,x,y,attr
0,0,4,A
1,0,7,A
2,0,16,A
3,0,32,B
4,0,35,B


In [0]:
# Подсчёт точности классификации
diff_df = classified_unclassified_data.merge(knn_data, left_on=['x','y'], right_on=['x','y'], suffixes=['_left', '_right'])
diff_df = diff_df.loc[diff_df['attr_left'] == diff_df['attr_right']]
accuracy = len(diff_df) * 100 / len(knn_unclassified_data)
print(f"Точность классификации: {round(accuracy, 2)}%")

Точность классификации: 66.57%